<a href="https://colab.research.google.com/github/limshaocong/SysBERT/blob/main/Sequence_classification_PROMISE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.3 MB/s 
     |████████████████████████████████| 4.2 MB 52.5 MB/s 
     |████████████████████████████████| 140 kB 61.4 MB/s 
     |████████████████████████████████| 1.1 MB 44.0 MB/s 
     |████████████████████████████████| 212 kB 62.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 127 kB 70.2 MB/s 
     |████████████████████████████████| 6.6 MB 52.1 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 271 kB 56.4 MB/s 
     |████████████████████████████████| 144 kB 42.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c6e9bc82-3781-50a8-1769-ae4439a82508)


# Import & Pre-process Data

Clone the repository containing the labeled requirements data.

In [107]:
! git clone https://github.com/limshaocong/SysBERT/

Cloning into 'Requirements'...
remote: Not Found
fatal: repository 'https://github.com/limshaocong/SysBERT/Requirements/' not found


This sequence classification task is performed using the labeled PROMISE dataset. The targte variable is denoted by the 'is_functional' column; 1 = functional requirement, 0 = non-functional requirement. The train, validation and test datasets are created by stratified sampling in a 70/15/15 ratio. The data is imported as a HuggingFace [Dataset](https://huggingface.co/docs/datasets/access) object for ease of downstream manipulation.

In [97]:
from datasets import load_dataset

train_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/train.csv'
val_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/val.csv'
test_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/test.csv'

promise = load_dataset(
    'csv',
    data_files = {
        'train': train_path,
        'val' : val_path,
        'test': test_path
        }
)

promise

Using custom data configuration default-8e27668ea1f53e98
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-8e27668ea1f53e98/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 437
    })
    val: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 94
    })
    test: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 94
    })
})

To transform natural language requirements into a BERT-compatible format, the text must first be tokenized. This is performed using a pre-trained tokenizer.

In [96]:
from transformers import AutoTokenizer

model_checkpoint = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = True, max_length = 128)

promise_tokenized = promise.map(encode, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [106]:
sample = promise['train'][0]

print(sample)
encode(sample)

{'reqs': 'All movies shall be streamed on demand at any time of the day.', 'is_functional': 1}


{'input_ids': [101, 1398, 5558, 4103, 1129, 20273, 1113, 4555, 1120, 1251, 1159, 1104, 1103, 1285, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [98]:
promise_tokenized

DatasetDict({
    train: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 437
    })
    val: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 94
    })
    test: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 94
    })
})

In [99]:
from transformers import DataCollatorWithPadding

batch_size = 16
output_col = 'is_functional'

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    return_tensors = 'tf'
)

batched_train_dataset = promise_tokenized['train'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

batched_val_dataset = promise_tokenized['val'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

batched_test_dataset = promise_tokenized['test'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

# Model Fine-tuning

In [108]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

tf.keras.utils.set_random_seed(1234)

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels = 2
)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [110]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________
